In [27]:
from scipy.io import arff
import numpy as np
data,meta = arff.loadarff("PhishingData.arff")
meta
data_ = data.astype(np.dtype([('SFH', 'int64'), ('popUpWidnow', 'int64'), ('SSLfinal_State', 'int64'), ('Request_URL', 'int64'), ('URL_of_Anchor', 'int64'), ('web_traffic', 'int64'), ('URL_Length', 'int64'), ('age_of_domain', 'int64'), ('having_IP_Address', 'int64'), ('Result', 'int64')]))

count_1 = 0; count_neg_1 = 0; count_0 = 0


partition_by_classes = {
    -1: [],
    0:[],
    1: []
}
neg_class = [np.array(point.item()) for point in data_ if point[-1] == -1]
zero_class = [np.array(point.item()) for point in data_ if point[-1] == 0]
one_class = [np.array(point.item()) for point in data_ if point[-1] == 1]
# for point in data_:
#     if point[-1] == 1:
#        count_1 += 1
#     elif point[-1] == -1:
#         count_neg_1 +=1
#     else:
#         count_0 +=1
# print(count_neg_1, count_0, count_1)
print(len(neg_class), len(zero_class), len(one_class))

702 103 548


In [28]:
k = 10

# Implementing K fold cross validation

def create_k_fold(data, k):
    k_subset = []
    start_point = 0
    for i in range(k):
        if i == k -1:
            k_subset.append(data[start_point:])
            continue
        rough_size = len(data) //k;
        subset = data[start_point:start_point + rough_size]
        k_subset.append(subset)
        start_point = start_point + rough_size
    
    return k_subset

k_neg_class = create_k_fold(neg_class, 10)
k_zero_class = create_k_fold(zero_class, 10)
k_one_class = create_k_fold(one_class, 10)

# getting train and test data from k partitioned data set

def get_train_test_data(i):
    test = k_neg_class[i] + k_one_class[i] + k_zero_class[i]
    train = []
    for j in range(len(k_neg_class)):
        if j != i:
            train = train + k_neg_class[j]
    for j in range(len(k_one_class)):
        if j != i:
            train = train + k_one_class[j]
    for j in range(len(k_zero_class)):
        if j != i:
            train = train + k_zero_class[j]
    
    return train, test

def count_class(c):
    class_1 = 0; class_0 = 0; class_neg = 0
    for i in c:
        if i[-1] == 0:
            class_0 +=1
        elif i[-1] == 1:
            class_1 +=1
        else:
            class_neg +=1
    return class_neg, class_0, class_1


In [29]:


def KClustering(train):
    k=3
    p_1 = train[0]
    p_2 = train[len(train)-1]
    p_3 = train[len(train)//2]
    p_1 = p_1[:-1]
    p_2 = p_2[:-1]
    p_3 = p_3[:-1]
    for i in range(50):
        c_1 = []; c_2 = []; c_3 = []
        for point in train:
            p = point
            d_1 = np.linalg.norm(p[:-1] - p_1)
            d_2 = np.linalg.norm(p[:-1] - p_2)
            d_3 = np.linalg.norm(p[:-1] - p_3)
            d_min = min(d_1,d_2,d_3)
            if d_min == d_1:
                c_1.append(p)
            elif d_min == d_2:
                c_2.append(p)
            else:
                c_3.append(p)
        c_1_no_label = [x[:-1] for x in c_1]
        c_2_no_label = [x[:-1] for x in c_2]
        c_3_no_label = [x[:-1] for x in c_3]
        p_1 = np.mean(c_1_no_label, axis = 0)
        p_2 = np.mean(c_2_no_label, axis = 0)
        p_3 = np.mean(c_3_no_label, axis = 0)
        if i % 10 == 0:
            print("$$$$$$$$$$$$$$$$$$$$$$ %s $$$$$$$$$$$$" % i)
            class_neg, class_0, class_1 = count_class(c_1)
            print("cluster #1: -1:%s 0:%s 1:%s" % (class_neg, class_0, class_1))
            class_neg, class_0, class_1 = count_class(c_2)
            print("cluster #2: -1:%s 0:%s 1:%s" % (class_neg, class_0, class_1))
            class_neg, class_0, class_1 = count_class(c_3)
            print("cluster #3: -1:%s 0:%s 1:%s" % (class_neg, class_0, class_1))
        
        


In [30]:
train, test = get_train_test_data(9)
KClustering(train)
class_neg, class_0, class_1 = count_class(train)
print("cluster #1: -1:%s 0:%s 1:%s" % (class_neg, class_0, class_1))

$$$$$$$$$$$$$$$$$$$$$$ 0 $$$$$$$$$$$$
cluster #1: -1:333 0:16 1:200
cluster #2: -1:46 0:56 1:239
cluster #3: -1:251 0:18 1:47
$$$$$$$$$$$$$$$$$$$$$$ 10 $$$$$$$$$$$$
cluster #1: -1:173 0:19 1:118
cluster #2: -1:31 0:31 1:279
cluster #3: -1:426 0:40 1:89
$$$$$$$$$$$$$$$$$$$$$$ 20 $$$$$$$$$$$$
cluster #1: -1:173 0:19 1:118
cluster #2: -1:31 0:31 1:279
cluster #3: -1:426 0:40 1:89
$$$$$$$$$$$$$$$$$$$$$$ 30 $$$$$$$$$$$$
cluster #1: -1:173 0:19 1:118
cluster #2: -1:31 0:31 1:279
cluster #3: -1:426 0:40 1:89
$$$$$$$$$$$$$$$$$$$$$$ 40 $$$$$$$$$$$$
cluster #1: -1:173 0:19 1:118
cluster #2: -1:31 0:31 1:279
cluster #3: -1:426 0:40 1:89
cluster #1: -1:630 0:90 1:486


In [23]:

# TRAINING GAUSSIAN MIXTURE MODEL

from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=3)
# train = [np.array(point.item()) for point in train]
train_no_label = [p[:-1] for p in train]

gmm.fit(train_no_label)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=3, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [26]:
# FITTING THE DATA AND PRINTOUT HOW MANY OF EACH CLASS IS CORRECTLY/INCORRECTLY MISCLUSTERED

c_1 = 0; c_2 = 0; c_3 = 0
c = 0

for i in range(630):
    p = train[i]
    if p[-1] == 0:
        c_1 +=1
    elif p[-1] == 1:
        c_2 +=1
    else:
        c_3 +=1
print(c_1,c_2,c_3)
    
labels = gmm.predict(train_no_label[:630])
# print(labels)
count_diff = 0
c_1 = 0; c_2 = 0; c_3 = 0
for i in range(630):
    label = labels[i]
    if label == 0:
        c_1 +=1
    elif label == 1:
        c_2 +=1
    else:
        c_3 +=1
print("all instances from Class 1 is labeled as")
print("cluster1: %s   cluster2: %s   cluster3: %s \n" %(c_1,c_2,c_3))
labels = gmm.predict(train_no_label[630:630+90])
# print(labels)
count_diff = 0
c_1 = 0; c_2 = 0; c_3 = 0
for i in range(90):
    label = labels[i]
    if label == 0:
        c_1 +=1
    elif label == 1:
        c_2 +=1
    else:
        c_3 +=1
print("all instances from Class 2 is labeled as")
print("cluster1: %s   cluster2: %s   cluster3: %s \n" %(c_1,c_2,c_3))
labels = gmm.predict(train_no_label[720:])
# print(labels)
count_diff = 0
c_1 = 0; c_2 = 0; c_3 = 0
for i in range(486):
    label = labels[i]
    if label == 0:
        c_1 +=1
    elif label == 1:
        c_2 +=1
    else:
        c_3 +=1
print("all instances from Class 3 is labeled as")
print("cluster1: %s   cluster2: %s   cluster3: %s \n" %(c_1,c_2,c_3))

0 0 630
all instances from Class 1 is labeled as
cluster1: 188   cluster2: 186   cluster3: 256 

all instances from Class 2 is labeled as
cluster1: 34   cluster2: 50   cluster3: 6 

all instances from Class 3 is labeled as
cluster1: 178   cluster2: 235   cluster3: 73 

